In [1]:
import glob
import re

In [2]:
%load_ext rpy2.ipython

/home/timp/miniconda3/envs/cpowgs/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Ok - trying to use prokka - roary to make a pangenome

Made a conda environment conda create --name cpowgs
Then conda install -c conda-forge -c bioconda -c defaults prokka
then
conda config --add channels r
conda config --add channels defaults
conda config --add channels conda-forge
conda config --add channels bioconda
conda install roary
then
conda install jupyterlab (I know, wrong order)
then
conda install raxml
then
conda install snp-sites
then
conda install pandas

In [10]:
%cd ~/Data/200404_trish/ecoli/

/mnt/d/Data/200404_trish/ecoli


ok - clean up the pilon assmeblies so that the headers don't have _pilon_ over and over and over

In [3]:
# !sed -i 's/_pilon//g' pilon/*fasta

Ok - I need to pull down the genbank files for Enterobacter cloacae: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
E. Coli: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
and K. Pneumo: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz

In [11]:
%mkdir refs
%cd refs
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff ecoli_ref.gb
%cd ..

/mnt/d/Data/200404_trish/ecoli/refs
--2020-04-05 15:43:59--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::12
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4011962 (3.8M) [application/x-gzip]
Saving to: ‘GCF_000240185.1_ASM24018v2_genomic.gbff.gz’

GCF_000240185.1_ASM 100%[===================>]   3.83M  --.-KB/s    in 0.1s    

2020-04-05 15:43:59 (33.5 MB/s) - ‘GCF_000240185.1_ASM24018v2_genomic.gbff.gz’ saved [4011962/4011962]

/mnt/d/Data/200404_trish/ecoli


Ok - now prokka

In [12]:
%rm -R prok
%rm -R gff
%mkdir gff
filelist=(glob.glob("pilon/*fasta"))
for myfile in filelist:
    tag=re.split('[\/\.]', myfile)[1]
    print(tag)
    !prokka --quiet --protein refs/ecoli_ref.gb --outdir prok/$tag --genus Escherichia --usegenus --prefix $tag --locustag $tag $myfile 
    %cp prok/$tag/*gff gff

ecoli_100
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok/ecoli_100/ecoli_100.gbf
[tbl2asn-forever] Correcting dates in prok/ecoli_100/ecoli_100.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 05-APR-2020
ecoli_102
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok/ecoli_102/ecoli_102.gbf
[tbl2asn-forever] Correcting dates in prok/ecoli_102/ecoli_102.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 05-APR-2020
ecoli_173
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok/ecoli_173/ecoli_173.gbf
[tbl2asn-forever] Correcting dates in prok/ecoli_173/ecoli_173.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 05-APR-2020
ecoli_174
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok/ecoli_174/ecoli_174.gbf

Ok - now let's try roary

Roary says first https://github.com/microgenomics/tutorials/blob/master/pangenome.md
you gotta organize the gff files

In [13]:
!roary -i 95 -f pan -e -n -p 8 gff/*gff

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.
Use of uninitialized value in require at (eval 931) line 1.


Counting snps from the multi-fa using snp-sites tool

In [14]:
!snp-sites -v pan/core_gene_alignment.aln >pan/core_gene_alignment.vcf

In [15]:
%%R 
library(tidyverse)

vcfs=read_tsv("pan/core_gene_alignment.vcf", comment="##")

##ok - for each sample column, need to look at delta for all other sample columns, then sum to get that square on the table
##Interesting names are names(vcfs)[10:22]

comp=as_data_frame(t(combn(names(vcfs[10:22]),2))) %>%
    mutate(val=colSums(abs(vcfs[V1]-vcfs[V2])))

comp=rbind(comp, data_frame(V1=comp$V2, V2=comp$V1, val=comp$val))

write_csv(spread(comp, V2, val), "pan/core_gene_alignment_count.csv")

R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)

R[write to console]: See spec(...) for full column specifications.



Ok - that generated a pan genome and core alignment file which is already getting me close.  I then ran it in geneious, but let me take a guess and try to run in raxml
some tips:
-p is a random parsimony seed - to make sure analysis does the same thing
-m GTRGAMMA - GTR GAMMA method
-x is the rapid bootstrapping seed
-N is the number of bootstrapops
-f a says do rapid bootstrap

In [16]:
%rm -R raxml
%mkdir raxml
%cd raxml
!raxmlHPC-PTHREADS -T 6 -f a -m GTRGAMMA -p 53243 -x 72341 -s ../pan/core_gene_alignment.aln -N 100 -n testrapid.tree
%cd ..

/mnt/d/Data/200404_trish/ecoli/raxml

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 5


This is RAxML version 8.2.12 released by Alexandros Stamatakis on May 2018.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
Kassian Kobert    (HITS)
David Dao         (KIT and HITS)
Sarah Lutteropp   (KIT and HITS)
Nick Pattengale   (Sandia)
Wayne Pfeiffer    (SDSC)
Akifumi S. Tanabe (NRIFS)
Charlie Taylor    (UF)


Alignment has 23487 distinct alignment patterns

Proportion of gaps and completely undetermined characters in this alignment: 3.33%

RAxML rapid